<a href="https://colab.research.google.com/github/RichardMathewsII/BODnet/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load necessary modules

In [2]:
!git clone https://github.com/RichardMathewsII/BODnet
# !git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'BODnet'...
remote: Enumerating objects: 5593, done.
remote: Counting objects: 100% (5593/5593), done.
remote: Compressing objects: 100% (1567/1567), done.
remote: Total 5593 (delta 3981), reused 5589 (delta 3980), pack-reused 0
Receiving objects: 100% (5593/5593), 8.68 MiB | 2.57 MiB/s, done.
Resolving deltas: 100% (3981/3981), done.


In [3]:
cd /content/BODnet/

/content/BODnet


In [4]:
# requirement already satisfied
!pip install numpy
!pip install tensorflow

In [5]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/BODnet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_overlap.o
In 

In [6]:
# done
import keras

Using TensorFlow backend.


In [9]:
# import python modules from keras_retinanet
import sys
sys.path.insert(0, '/content/BODnet')

# done
!pip install keras_resnet

# done
# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

In [8]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import os

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
setup_gpu(gpu)

In [0]:
# upload data into snapshots and check that it's there
os.chdir('/content/BODnet/snapshots')
!ls

In [11]:
# check current working directory to confirm it's in snapshots
os.getcwd()

'/content/BODnet/snapshots'

## Load RetinaNet model

In [12]:
# upload trained model into snapshots
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
path = os.getcwd()
model_path = os.path.join(path, 'resnet50_coco_best_v2.1.0.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
# NOTE: OSError: Unable to open file (truncated file: eof = 27262976, sblock->base_addr = 0, stored_eof = 152662144)
# This error arises when the file was corrupted upon uploading. Delete the file and reupload.

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
# model = models.convert_model(model)

# print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [13]:
!ls

resnet50_coco_best_v2.1.0.h5


## Run detection on example

In [0]:
cap = cv2.VideoCapture('/content/BODnet/snapshots/busy_street.mp4')

if not cap.isOpened():
    print("Error opening video file")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('busy_street_ann2.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        # load image
        # image = read_image_bgr(frame)

        # copy to draw on
        draw = frame.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

        # preprocess image for network
        image = preprocess_image(frame)
        image, scale = resize_image(image)

        # process image
        start = time.time()
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        # print("processing time: ", time.time() - start)

        # correct for image scale
        boxes /= scale

        # visualize detections
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            # scores are sorted so we can break
            if score < 0.5:
                break
        
            color = label_color(label)
    
            b = box.astype(int)
            draw_box(draw, b, color=color)
    
            b = np.array(box).astype(int)
            # plt.text(b[0], b[1], s=labels_to_names[label], fontsize = 12, color='white', verticalalignment='top')
            # caption = "{} {:10.5f}".format(labels_to_names[label], score)
            # draw_caption(draw, b, caption)
            cv2.putText(draw, labels_to_names[label], (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 0), 2)
            cv2.putText(draw, labels_to_names[label], (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 1)
        out.write(draw)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [0]:
from PIL import Image
import datetime
import matplotlib.patches as patches
import random
img_size = 416
# load image and get detections
img_path = "test_img.jpg"
prev_time = time.time()
img = Image.open(img_path)
boxes, scores, labels = model.predict_on_batch(np.expand_dims(img, axis=0))
inference_time = datetime.timedelta(seconds=time.time() - prev_time)
print ('Inference Time: %s' % (inference_time))
# Get bounding-box colors
cmap = plt.get_cmap('tab20b')
colors = [cmap(i) for i in np.linspace(0, 1, 20)]
unique_labels = np.unique(labels)
n_cls_preds = len(unique_labels)
# bbox_colors = random.sample(colors, n_cls_preds)

img = np.array(img)
plt.figure()
fig, ax = plt.subplots(1, figsize=(12,9))
ax.imshow(img)
pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
unpad_h = img_size - pad_y
unpad_w = img_size - pad_x
if scores is not None:
    # browse detections and draw bounding boxes
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < 0.5:
          break
        b = box.astype(int)
        b = np.array(box).astype(int)
        x1, y1, x2, y2 = b[0], b[1], b[2], b[3]
        box_h = ((y2 - y1) / unpad_h) * img.shape[0]
        box_w = ((x2 - x1) / unpad_w) * img.shape[1]
        y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
        x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]
        # color = bbox_colors[int(np.where(
        #     unique_labels == int(label))[0])]
        bbox = patches.Rectangle((x1, y1), box_w, box_h,
             linewidth=2, edgecolor=(0.1, 0.2, 0.5), facecolor='none')
        ax.add_patch(bbox)
        plt.text(x1, y1, s=labels_to_names[label], 
                color='white', verticalalignment='top',
                bbox={'color': (0.1, 0.2, 0.5), 'pad': 0})
plt.axis('off')
# save image
# plt.savefig(img_path.replace(".jpg", "-det.jpg"),        
#                  bbox_inches='tight', pad_inches=0.0)
plt.show()

In [0]:
import colorsys
import random
N = len(labels_to_names)
HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in range(N)]
RGB_tuples = list(map(lambda x: tuple(255*np.array(colorsys.hsv_to_rgb(*x))), HSV_tuples))
random.shuffle(RGB_tuples)


In [0]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.physical_device_desc for x in local_device_protos if x.device_type == 'GPU']

GPU = get_available_gpus()[-1][17:33]
print(GPU)

Tesla T4, pci bu


In [0]:
os.getcwd()

'/content/keras-retinanet/snapshots'

In [0]:
from google.colab.patches import cv2_imshow
cap = cv2.VideoCapture('/content/keras-retinanet/snapshots/test.mp4')

counter = 0
sum_time=0
while(True):
    ret, draw = cap.read()
    if not ret:
        break
    bgr = cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)
    
    # preprocess image for network
    image = preprocess_image(bgr)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    t = time.time() - start
#     print("processing time: ", t)

    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < 0.5:
            continue
        color = label_color(label)
    
        b = box.astype(int)
        draw_box(draw, b, color=color)
    
        # caption = "{} {:.3f}".format(labels_to_names[label], score)
        # draw = draw_caption(draw, b, caption)
        # cv2.imwrite('/tmp/img%08d.jpg'%counter,draw)
    cv2_imshow(draw)
    if cv2.waitKey(1)  & 0xFF == ord('q'):
        break
    counter=counter+1
    sum_time+=t

cap.release()
cv2.destroyAllWindows()


In [0]:
detections.shape

(1, 300)